**The primary purpose of this notebook is to test the OpenEphys sorting "pipeline".
Specifically, after defining your pathnames appropriately, this notebook will use SpikeInterface to perform spike sorting, analyze the sorting quality, and exports the results in "Phy" format. This output is then passed into pyramid to extract behavioral data, eye data, and spiking data to create a "TrialFile" which can be easily converted in Matlab to a user-friendly format, like FIRA, to perform subsequent analyses.**

Notes: 
1) MAKE SURE YOUR KERNEL (top right corner of the notebook in VS Code) IS "gold_pipelines (Python X.XX.X)
2) This should be run with VS code when your top level folder is /Lab_Pipelines/experiments/aodr/
3) Can't find a module/package? Check if you have a jupyter conflict. If your jupyter notebook is local, it's looking for resources based on the local jupyter paths. We want our jupyter notebook to be working in the same environment we're using (gold_pipelines). This might pose problems for other work you do with jupyter, but I'm not sure yet (you might want to consider only installing jupyter in python environments). Check which jupyter you're using by typing "which jupyter" in a terminal with the gold_pipelines environment activated. If it references a jupyer that is located in ".../anaconda3/envs/gold_pipelines" then you should be okay. If it's something else then you should try uninstalling jupyter:

```python
conda remove jupyter jupyter-client jupyter-console jupyter-core

conda uninstall jupyter_core nbformat nbconvert notebook
```

Then reinstall the gold_pipelines environment.

In [1]:
import spikeinterface.widgets as sw
from AODR_session_sorters import OpenEphysSessionSorter as OES
from pyramid import cli
import pandas as pd
import sys, os
%matplotlib ipympl

Below, we define the relevant paths. You should change these to match the corresponding paths on your machine. If you encounter issues while loading/sorting the file, chances are that your computer is having a hard time accessing the files from cloud storage. If that is the case, you need to transfer your files to a local directory, and also save the files to a local directory.

In [2]:
# Experiment directory top level - should contain the subfolders like ecodes, python, matlab.
expDir = "/Users/lowell/Documents/GitHub/Lab_Pipelines/experiments/aodr"
sessDir = "MrM_2025-10-03_13-29-04" #"MrM_2025-09-26-Trimmed"
os.chdir(expDir) # regular python files will run from the first open folder in your tree, but notebooks will work from wherever the notebook is stored.
# Directory where the raw files are stored to be converted/sorted
dataSearchPath = "/Users/lowell/Library/CloudStorage/Box-Box/GoldLab/Data/Physiology/AODR/dlPFC/MrM/Raw/Neuronal/SingleEL/"
# Where the rules for ecodes are stored
pyramidSearchPath = expDir+"/ecodes"
# Conversion specifications
convertSpecs = expDir+"/AODR_experiment_LC.yaml"
# Base directory to save the output files from pyramid (hdf5 files)
baseSaveDir = "/Users/lowell/Library/CloudStorage/Box-Box/GoldLab/Data/Physiology/AODR/dlPFC/MrM/Converted/Pyramid/Neuronal/SingleEL/"
# The Open Ephys experiment name
currentFile = "experiment1.nwb"
# Full directory to save the output files from pyramid (hdf5 files)
trialFileOutputName = baseSaveDir+sessDir+".hdf5"
# Directory to save the output files from sorting
sorted_out = '/Users/lowell/Library/CloudStorage/Box-Box/GoldLab/Data/Physiology/AODR/dlPFC/MrM/Sorted/SingleEL/'+sessDir+"/"

sys.path.append(expDir+"/python") # to make sure pyramid can access the custom collectors/enhancers/functions?

Below, we run the sorter/analyzer. With ~4 channels of neural data and a 90 min recording, this should take between 10-20 minutes. The step_names variable defines the sequential processing steps that are defined as methods in AODR_session_sorters OpenEphysSessionSorter class. If you use the step "open_sigui", a gui window will open that allows to interact with the sorting results. Close the gui to continue to the next step and save the output.

In [ ]:
sorter = OES(session_dir=dataSearchPath+sessDir+"/", channel_names=[1],
             out_folder=sorted_out,
             stream_name='acquisition_board',
             overwrite_timestamps=True,             # This will overwrite the first sample time in the phy params.py file since the first sample number has non-zero time
             sorter_name='single_channel',
             step_names=[
                 'clean_tree',                      # This removes everything from the output folder!!!
                 'read_data',                       # Has to be included to do anything else
                 'set_single',
                 'bandpass',
                 'single_ch_sorter_and_analyzer',     # Runs one spike sorter which is an optional parameter (default spikecyrcus2) and one analyzer
                 'open_sigui',                      # Opens a gui to view the sorting results to help with manual refinement
                 'export_to_phy'                    # Currently pyramid supports Phy output or plexon
             ])

clean_tree ...OK
read_data ...OK
set_single ...OK
bandpass ...OK
Warning! The recording is already filtered, but mountainsort5 filter is enabled
filtering
whitening
write_binary_recording 
n_jobs=12 - samples_per_chunk=29,999 - chunk_memory=117.18 KiB - total_memory=1.37 MiB - chunk_duration=1.00s (999.97 ms)


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

write_binary_recording:   0%|          | 0/5932 [00:00<?, ?it/s]

ERROR:tornado.general:SEND Error: Host unreachable


Spike interface has a lot of widgets availble to use with python jupyter notebooks. These widgets are interactive and useful, and the below code will test whether they are working appropriately on your system.

In [ ]:
w_ts_filt = sw.plot_spikes_on_traces(sorter.sorting_analyzer, channel_ids=[1], backend="ipywidgets")

Now, run Pyramid on the corresponding output to create a TrialFile. Make sure that the phy_reader.params_file points to the appropriate directory based on those you have defined above.

In [ ]:
cli.main(["convert", 
        "--trial-file", trialFileOutputName, 
        "--search-path", pyramidSearchPath, 
        "--experiment", convertSpecs, 
        "--readers", 
        "ttl_reader.session_dir="+dataSearchPath+sessDir,
        "message_reader.session_dir="+dataSearchPath+sessDir,
        "gaze_x_reader.session_dir="+dataSearchPath+sessDir,
        "gaze_y_reader.session_dir="+dataSearchPath+sessDir,
        "pupil_reader.session_dir="+dataSearchPath+sessDir,
        "phy_reader.params_file="+sorted_out+"/phy/params.py"])